预测用户消费的案例

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('ml').getOrCreate()

In [2]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [3]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [5]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

找出数据集中的变量和结果列，VectorAssembler 的作用就是把多个列合并成一个向量列

In [8]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'],
                            outputCol='features') 

In [9]:
output = assembler.transform(data)

In [12]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

观察上面的输出，最后一列是新生成的

In [13]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [15]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

到这里数据就基本准备好了，有了输入和输出，下面就是进行训练和对比, 属于固定流程

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                347|
|   mean| 499.46038404044936|
| stddev|  78.94634452763194|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                153|
|   mean|  498.9821298513693|
| stddev|  80.40393115755697|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
lr = LinearRegression(labelCol='Yearly Amount Spent', featuresCol='features')

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-13.327576175648005|
|-0.6110925278131276|
|-18.638647756921387|
|-13.932817469975532|
| 1.6338815187692717|
|  18.47519554496563|
| 2.4880259509701546|
| -5.381003935744502|
| 1.6814206942801206|
| -5.912175921768949|
| -6.166100136257114|
| 2.2754842300694804|
|-19.252699091048044|
| -15.57702846967527|
|-2.9614074065203795|
| -3.062980592615702|
|-5.4168013306323815|
| -3.350577624566995|
|-12.238945029812044|
|-18.158502207805498|
+-------------------+
only showing top 20 rows



In [25]:
test_results.rootMeanSquaredError

9.143636578591085

In [26]:
test_results.r2

0.9869823920009901

从上面的方差可以看到，误差率还可以。 下面使用模型来进行预测

In [27]:
unlabel_data = test_data.select('features')

In [28]:
pre_res = lr_model.transform(unlabel_data)

In [29]:
pre_res.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 333.2564459788416|
|[30.5743636841713...| 442.6755062858788|
|[30.8162006488763...| 284.7249887053904|
|[31.0662181616375...| 462.8661106776499|
|[31.3091926408918...|431.08683632116436|
|[31.3123495994443...|  445.116222482975|
|[31.3584771924370...|492.68792449850525|
|[31.4252268808548...| 536.1477225905064|
|[31.4459724827577...|483.19554424084845|
|[31.5147378578019...|495.72466391823036|
|[31.5257524169682...|  450.131726946139|
|[31.5316044825729...| 434.2401214992931|
|[31.5702008293202...| 565.1981912324529|
|[31.5741380228732...| 559.9863006302621|
|[31.5761319713222...| 544.1879913958487|
|[31.7216523605090...|350.83990722448834|
|[31.7656188210424...| 501.9708829662395|
|[31.8530748017465...|462.63570108691897|
|[31.8648325480987...| 452.1302255066257|
|[31.9048571310136...|492.10835963062164|
+--------------------+------------